In [1]:
import pandas as pd
import plotly.express as px
import dash
import matplotlib.pyplot as plt
import wget

from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
from dash import html
from dash import dcc 

In [2]:

# Read the spacex data into pandas dataframe
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
dataset = wget.download(url)
spacex_df = pd.read_csv(dataset)





100% [................................................................................] 2476 / 2476

In [3]:
spacex_df.dtypes

Unnamed: 0                    int64
Flight Number                 int64
Launch Site                  object
class                         int64
Payload Mass (kg)           float64
Booster Version              object
Booster Version Category     object
dtype: object

In [4]:
app = JupyterDash(__name__)

# marker_points will be a dictionary to be used for marks on the App's RangeSlider
marker_points = {}
for i in range(10):
    marker_points[i*1000] = str(i*1000)
    
"""
Here the app layout is created, with title, a division for choosing the site location,
    a division for showing a pie chart for success rates at that site,
    a division for selecting a range of payload (from 0 to 10,000 kg),
    and a division for showing a scatter plot for comparing a launch's payload to if it was successful
    
*NOTE that class = 1 means successful, and class = 0 means unsuccesful

"""

app.layout = html.Div(children=[html.H1("SpaceX Launch Records Dashboard", style={'text-align': 'center'}),
                                html.Div(dcc.Dropdown(id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},                                        
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                        ],
                                    value='ALL',
                                    placeholder="Select a Launch Site here",
                                    searchable=True
                                               )
                                        ),
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                
                                html.Div(dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    marks=marker_points,
                                    value=[0, 10000])
                                        ),       
                                html.Div(dcc.Graph(id="scatter-chart")),
                                ])

#This first function is for creating the pie chart

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):

    #Here if 'ALL' is chosen for site location, the figure will compare success rates between sites
    
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', 
            title='Total Successful Launches by Site')
        return fig   
    
    #If a site is chosen instead, the pie chart will show the ratio between successful and unsuccesful launches 
    else:
        filtered_df = spacex_df.loc[spacex_df['Launch Site'] == entered_site]      
        fig = px.pie(filtered_df, names='class', 
            title=entered_site + ' success rate')
        return fig 
    
    
#This next function is for creating the scatter plot

@app.callback(Output(component_id='scatter-chart', component_property='figure'), 
               Input(component_id="payload-slider", component_property="value"),
                Input(component_id='site-dropdown', component_property='value'))
              
def get_scatter(payload_vals, entered_site):
    
    #This time the dataframe used for the graphs is restricted depending on the input of the range slider
    
    low_df = spacex_df.loc[(spacex_df['Payload Mass (kg)'] > payload_vals[0])]
    high_df = low_df.loc[(spacex_df['Payload Mass (kg)'] < payload_vals[1])]
    
    
    #This code is similar to that used in the get_pie_chart function, 'ALL' compares all site locations to each other
    if entered_site == 'ALL':
        final_df = high_df
        fig = px.scatter(final_df, y="class", x="Payload Mass (kg)", color="Booster Version")
        return fig
    
    #Likewise, when a particular site is selected, successful to unsuccessful launches will be compared based on payload
    else:
        filtered_df = high_df.loc[spacex_df['Launch Site'] == entered_site] 
        final_df = filtered_df
        fig = px.scatter(final_df, y="class", x="Payload Mass (kg)", color="Booster Version")
        return fig

#Run the app
if __name__ == '__main__':
    app.run_server(mode='inline')